#Código responsável por ler o json e criar os superpixels das áreas demarcadas.

Cria o link para o google drive

In [5]:
!pip3 install labelme
!pip3 install tqdm
!pip3 install shapely

  Using cached matplotlib-3.2.2-cp38-cp38-manylinux1_x86_64.whl (12.4 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.4
    Uninstalling matplotlib-3.3.4:
      Successfully uninstalled matplotlib-3.3.4
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

superannotate 1.7.2 requires matplotlib>=3.3.1, but you'll have matplotlib 3.2.2 which is incompatible.
You should consider upgrading via the '/home/diego/anaconda3/envs/sklearn/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/diego/anaconda3/envs/sklearn/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 1.1 MB 95 kB/s eta 0:00:011
You should consider upgrading via the '/h

In [6]:

#from google.colab import drive
#drive.mount('/content/gdrive')
 
import sys

base_colab=''
sys.path.append(base_colab)

In [7]:
import json
from types import SimpleNamespace
import re
import glob
import os
import labelme
import base64
import PIL 
from labelme import utils
import os.path as osp
import numpy as np
import PIL.Image
import PIL.ImageDraw
import threading
import time

#https://stackoverflow.com/questions/54506079/slic-segmentation-in-skimage
#https://stackoverflow.com/questions/26237580/skimage-slic-getting-neighbouring-segments

import skimage
from skimage.segmentation import slic
import io
import cv2
from skimage.measure import regionprops
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from tqdm import tqdm

import os
DATASET_LIST = os.listdir(base_colab+'data')
print(DATASET_LIST)
def getRAMinfo( ):
        p = os.popen('free')
        i = 0
        while 1:
            i = i + 1
            line = p.readline()
            if i==2:
                return(line.split()[1:4])


def check_class_and_save(DATASET,DATASET_SUPERPIXEL,obj,filename_no_ext,filename):
                    # Skimage
                    src = cv2.imread(DATASET+'/'+filename)
                    segments = skimage.segmentation.slic(src,n_segments = 4000, compactness=10.0, sigma = 5,enforce_connectivity=True)
                    segments_ids = np.unique(segments)
        
                    # centers
                    #centers = np.array([np.mean(np.nonzero(segments==i),axis=1) for i in segments_ids])
                    
                    #print("\nCheck all superpixels and separate each class\n")
                    #with tqdm(total=len(segments_ids)) as pbar:
                    for idx in segments_ids:

                        #print(idx)
                        center = np.mean(np.nonzero(segments==idx),axis=1)
                        # Create a mask, painting black all pixels outside of segment and white the pixels inside.
                        mask_segment = np.zeros(src.shape[:2], dtype="uint8")
                        mask_segment[segments == idx] = 255
                        size_segment = mask_segment[segments == idx].size
                        segment = src.copy()
                        segment = cv2.bitwise_and(segment, segment, mask=mask_segment)

                        # Get the countours around the segment
                        contours, _  = cv2.findContours(mask_segment,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]
                        m = -1
                        max_contour = None
                        for cnt in contours:
                          if (len(cnt) > m):
                            m = len(cnt)
                            max_contour = cnt

                        # Get the rectangle that encompasses the countour
                        x,y,w,h = cv2.boundingRect(max_contour)
                        segment = segment[y:y+h, x:x+w]
        
                        #cv2.imshow(str(idx),segment)
                      
                        find=False
                        
                        for ob in obj['shapes']:
                          cx=int(x+w/2)
                          cy=int(y+h/2)
                          path=DATASET_SUPERPIXEL+'/'+ob["label"]                                  
                                                      
                          point = Point(int(cx),int( cy))
                          polygon = Polygon(ob["points"])
    
                          if(polygon.contains(point)):
      
                            cv2.imwrite(path+"/"+folder_base+"_"+filename_no_ext+"_"+str(idx)+".jpg",segment)
                            find=True
                            break
                            #DATASET_SUPERPIXEL+'/'+ob["label"])
                        #if(find==False):
                            #print("background")
                            #save_image(DATASET_SUPERPIXEL+"/background/"+folder_base+"_"+filename_no_ext+"_"+str(idx)+".jpg",segment) 
                            #cv2.imwrite(DATASET_SUPERPIXEL+"/background/"+filename_no_ext+"_"+str(idx)+".jpg",segment)   
                      
def save_image(location,img):
   cv2.imwrite(location,img)
#pip install labelme==3.16.7
##location from dataset



for cont,folder_base in enumerate(DATASET_LIST):
  if(cont<=9):
      continue        
       
  DATASET=base_colab+"data/"+folder_base+"/"


      

  DATASET_SUPERPIXEL=base_colab+"slic/4000"
  #find all json to read
  json_list = glob.glob(os.path.join(DATASET,'*.json'))
  print(json_list)
 
  for file_name in json_list:
          print(file_name)
          #read a file json
          with open(file_name, "r") as read_file:
            data=json.load(read_file)
            #print(data["_via_attributes"])
            try:
             lis_class=data["_via_attributes"]["region"]["type"]["options"]
             print(data["_via_attributes"]["region"]["type"]["options"])
            except:             
             lis_class=data["_via_attributes"]["region"]["Type"]["options"]
             print(data["_via_attributes"]["region"]["Type"]["options"])
            #add manual names
            #lis_class={'01':'Carex_riparia_live', '02':'Carex_riparia_dead', '03':'Phalaris_arundinacea', '04':'Phragmites_australis_live', '05': 'Phragmites_australis_dead', '051': 'Phragmitea_australis_mix_l_and_d', '011': 'Carex_spec', '06': 'Urtica_dioica','08': 'Typha_live','081': 'Typha_dead','10': 'Oenanthe_aquatica','11': 'Nymphaeidae','12':'Arctium_spec','13': 'Salix_alba','14': 'Cirsium_arvense','100': 'Sonstiges'}

            path=DATASET_SUPERPIXEL+'/background'
            print(lis_class)
            try:
                os.mkdir(path) 
            except OSError:
                print ("Creation of the directory %s failed" % path)
            else:
                print ("Successfully created the directory %s " % path)

            for key, value  in lis_class.items():
                            
                  path=DATASET_SUPERPIXEL+'/'+value                                 
                  try:
                      os.mkdir(path) 
                  except OSError:
                      print ("Creation of the directory %s failed" % path)
                  else:
                      print ("Successfully created the directory %s " % path)
            print("Processing files in json: "+str(file_name))      
            threads=[]  
            for nick in data["_via_img_metadata"]:
                #Capture name of image
                filename=data["_via_img_metadata"][nick]["filename"]
                print(data["_via_img_metadata"][nick])
                print("\nStart process in file:"+filename) 
                filename_no_ext=os.path.splitext(filename)[0]
                obj={}
                obj["version"]="3.16.7"
                obj["flags"]={}
                
                obj["shapes"]=[]
                obj["imagePath"]=filename
                image=None
                try: 
                   image = PIL.Image.open(DATASET+'/'+filename)
                except:
                   print("File not found:"+filename) 
                   continue
                width, height = image.size 
                obj["imageHeight"] = height
                obj["imageWidth"] = width
                #transform image in string
                img = labelme.LabelFile.load_image_file(DATASET+'/'+filename)
                image_data = base64.b64encode(img).decode('utf-8')
                obj["imageData"]=image_data   
                # get regions of json with coordinates inside of file
                for region  in data["_via_img_metadata"][nick]["regions"]:
                  #print(region["region_attributes"]["name"])  
                  #class_name= region["region_attributes"]["name"]
                  try:
                    type_name= region["region_attributes"]["type"]
                  except:
                    type_name= region["region_attributes"]["Type"]
                  class_name=""
                  print("Chaves")
                  print(type_name.keys())  
                  for key, value in type_name.items()  :
                      if(value==True):  
                       class_name=lis_class[key]
                       break
                  #if class_name!="":
                    #continue
                  if class_name=="":
                    print("Not found class name")
                    print(region["region_attributes"])
                    print(type_name)
                    try:
                     type_name= region["region_attributes"]["name"]
                     print("Found in attribute  name")
                     print(region["region_attributes"])
                     for key, value in type_name.items()  :
                      if(value==True):  
                       class_name=lis_class[key]
                       break 
                    except:
                     print("falhou")
                  #print((class_name+"-"+str(type_name))) 
                  print(region["shape_attributes"]["name"])
                  if (region["shape_attributes"]["name"]=="polygon"):
                    points_y= region["shape_attributes"]["all_points_y"]
                    points_x= region["shape_attributes"]["all_points_x"]
                  elif  region["shape_attributes"]["name"]=="rect" :
                    
                    y=region["shape_attributes"]["y"]
                    x=region["shape_attributes"]["x"]
                    w=region["shape_attributes"]["width"]
                    h=region["shape_attributes"]["height"]
                    points_y=[y,y,y+h,y+h]
                    points_x=[x,x+w,x+w,x]
                  else:
                    print("Shape undefined of coordinates: "+region["shape_attributes"]["name"])
                  #create object
                  obj_mark={"label":class_name}
                  obj_mark["lineColor"]=None
                  obj_mark["fillColor"]=None

                  obj_mark["shape_type"]= "polygon"
              
                  obj_mark["points"]=[]
                      
                  for i, y in enumerate(points_y):
                      vet=[points_x[i],y]
                      obj_mark["points"].append(vet)
                  obj['shapes'].append(obj_mark)
                
                with open( DATASET+'/'+filename.split('.')[0]+".json" , "w" ) as write:
                      json.dump( obj , write )
                th = threading.Thread(target=check_class_and_save,args=(DATASET,DATASET_SUPERPIXEL,obj,filename_no_ext,filename))
                #threads.append(th)   //nao gera os slics se tiver comentada
                #th.start()
 

     
            # Output is in kb, here I convert it in Mb for readability
            RAM_stats = getRAMinfo()
            RAM_total = round(int(RAM_stats[0]) / 1000,1)
            RAM_used = round(int(RAM_stats[1]) / 1000,1)
            print("RAM Total : "+str(RAM_total))
            print("RAM Used : "+str(RAM_used))
            print("Wait a moment, the threads are processing "+str(len(threads)) +" images, it may be delayed depending on the size or quantity of the images!")
          
            print("\nThreads of save superpixels in Google Drive\n")    
            with tqdm(total=len(threads)) as ppbar:
             for t in threads:
                    t.start()
                    time.sleep(10)
                    if((RAM_total)<34000):#se menor que 10gb
                        RAM_stats = getRAMinfo()
                        RAM_used = round(int(RAM_stats[1]) / 1000,1)
                        if((RAM_total-RAM_used)<19000):
                            t.join()
                    ppbar.update(1)
             ppbar.close()
            for t in threads:
                t.join() 
            del threads
               
   

['Auswahl_183_2019-07-30_1', 'Auswahl_259_2019_07_30', 'Auswahl_807_2019_07_04', 'Auswahl_1189_2019_07_03', 'Auswahl_190_2019_07_30', 'Auswahl_188_2019_07_08', 'Auswahl_236_2019_07_30', 'Auswahl_1604_2019_07_31', 'Auswahl_808_2019_07_09', 'Auswahl_258_2019_07_30', 'Auswahl_1599_2019_06_24', 'Auswahl_1598_2019_06_24', 'Auswahl_183_2019-07-30_2', 'Auswahl_275_2019_06_28', 'Auswahl_Sonstiges', 'Auswahl_187_2019_07_30', 'Auswahl_607_2019_07_30', 'Auswahl_272_2019_06_28', 'Auswahl_287_2019_07_30', 'Auswahl_235_2019_07_30', 'Auswahl_2484_2019-06-25']
['data/Auswahl_1599_2019_06_24/1599-2019_06_24.json']
data/Auswahl_1599_2019_06_24/1599-2019_06_24.json
{'01': 'carex riparia live', '03': 'Phalaris arundinacea'}
{'01': 'carex riparia live', '03': 'Phalaris arundinacea'}
Creation of the directory slic/4000/background failed
Creation of the directory slic/4000/carex riparia live failed
Creation of the directory slic/4000/Phalaris arundinacea failed
Processing files in json: data/Auswahl_1599_201

Chaves
dict_keys(['01'])
polygon
Chaves
dict_keys(['01'])
polygon
Chaves
dict_keys(['01'])
polygon
{'filename': '100_0005_0095.JPG', 'size': 8642140, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [2317, 2348, 2385, 2444, 2475, 2478, 2463, 2384, 2351], 'all_points_y': [1056, 1096, 1111, 1094, 1073, 1050, 1033, 1031, 1036]}, 'region_attributes': {'name': 'vegetation', 'type': {'03': True}}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [1503, 2677, 2914, 2976, 3058, 2161, 1251, 1134], 'all_points_y': [1959, 1965, 2422, 2721, 3078, 3223, 3154, 3004]}, 'region_attributes': {'name': 'vegetation', 'type': {'01': True}}}], 'file_attributes': {}}

Start process in file:100_0005_0095.JPG


0it [00:00, ?it/s]

Chaves
dict_keys(['03'])
polygon
Chaves
dict_keys(['01'])
polygon
RAM Total : 32824.8
RAM Used : 4559.8
Wait a moment, the threads are processing 0 images, it may be delayed depending on the size or quantity of the images!

Threads of save superpixels in Google Drive

['data/Auswahl_1598_2019_06_24/1598_2019_06_24 (3)_corrected.json']
data/Auswahl_1598_2019_06_24/1598_2019_06_24 (3)_corrected.json
{'13': 'Salix cinerea', '08': 'Typha live'}
{'13': 'Salix cinerea', '08': 'Typha live'}
Creation of the directory slic/4000/background failed
Creation of the directory slic/4000/Salix cinerea failed
Creation of the directory slic/4000/Typha live failed
Processing files in json: data/Auswahl_1598_2019_06_24/1598_2019_06_24 (3)_corrected.json
{'filename': '100_0001_0049.JPG', 'size': 8583857, 'regions': [], 'file_attributes': {}}

Start process in file:100_0001_0049.JPG
File not found:100_0001_0049.JPG
{'filename': '100_0001_0056.JPG', 'size': 8519689, 'regions': [{'shape_attributes': {'name': 

Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['08'])
polygon
Chaves
dict_keys(['08'])
polygon
Chaves
dict_keys(['08'])
polygon
Chaves
dict_keys(['08'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['13'])
polygon
Chaves
dict_keys(['08'])
polygon
Chaves
dict_keys(['08'])
polygon
{'filename': '100_0001_0057.JPG', 'size': 8508358, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [408, 409, 595, 865, 787, 698, 657, 578], 'all_points_y': [355, 355, 243, 480, 532, 506, 543, 523]}, 'region_attributes': {'name': 'vegetation', 'type': {'13': True}}}, {'shape_attributes

...

{'filename': '100_0009_0105.JPG', 'size': 8624606, 'regions': [], 'file_attributes': {}}

Start process in file:100_0009_0105.JPG
{'filename': '100_0009_0124.JPG', 'size': 8656297, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [631, 631, 725, 836, 980, 1017, 1025, 1080, 1102, 1140, 1154, 1135, 1078, 857, 738, 712, 726, 833, 849], 'all_points_y': [511, 513, 560, 566, 429, 373, 323, 310, 254, 182, 121, 72, 44, 23, 15, 45, 107, 220, 267]}, 'region_attributes': {'name': 'Vegetation', 'type': {'051': False, '04': True}, 'image quality': {'good_ill': True}}}], 'file_attributes': {}}

Start process in file:100_0009_0124.JPG
Chaves
dict_keys(['051', '04'])
polygon
{'filename': '100_0009_0157.JPG', 'size': 8505104, 'regions': [], 'file_attributes': {}}

Start process in file:100_0009_0157.JPG
{'filename': '100_0009_0172.JPG', 'size': 8543996, 'regions': [], 'file_attributes': {}}

Start process in file:100_0009_0172.JPG
{'filename': '100_0006_0001.JPG', 'size': 8421273, '

Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
{'filename': '100_0006_0039.JPG', 'size': 8522999, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [2366, 2367, 2806, 2133, 2061], 'all_points_y': [3, 2, 3, 423, 280]}, 'region_attributes': {'name': 'Vegetation', 'type': {'051': False, '04': True}, 'image quality': {'good_ill': True}}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [1398, 1435, 1873, 2002, 1916, 1730, 1651], 'all_points_y': [550, 720, 469, 304, 289, 416, 471]}, 'region_attributes': {'name': 'Vegetation', 'type': {'051': False, '04': True}, 'image quality':

Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
{'filename': '100_0006_0077.JPG', 'size': 8751231, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [414, 659, 742, 825, 755, 629], 'all_points_y': [554, 262, 231, 357, 505, 564]},

0it [00:00, ?it/s]

Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
Chaves
dict_keys(['051', '04'])
polygon
RAM Total : 32824.8
RAM Used : 4612.5
Wait a moment, the threads are processing 0 images, it may be delayed depending on the size or quantity of the images!

Threads of save superpixels in Google Drive

